In [ ]:
!pip install opendatasets

In [ ]:
import pandas as pd
import numpy as np
import opendatasets as od
od.download("https://www.kaggle.com/competitions/playground-series-s3e3/overview")

Skipping, found downloaded files in "./playground-series-s3e3" (use force=True to force download)


# Importing Data

In [ ]:
train_df = pd.read_csv("/content/playground-series-s3e3/train.csv",index_col = "id")


(1677, 34)

# # Data manipulation

Checking for numerical Data

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
cor =train_df.corr()
plt.subplots(figsize=(20,15))
sns.heatmap(cor,annot = True)

The Columns-
'Age','Education','EnvironmentSatisfaction','JobInvolvement', 'JobLevel','JobSatisfaction','MonthlyIncome','RelationshipSatisfaction', 'StockOptionLevel','TotalWorkingYears','YearsAtCompany', 'YearsInCurrentRole','YearsWithCurrManager'

Have a high correlation so we will use them to predict the data.


In [ ]:
input_col = ['Age','Education','EnvironmentSatisfaction','JobInvolvement', 'JobLevel','JobSatisfaction','MonthlyIncome','RelationshipSatisfaction', 'StockOptionLevel','TotalWorkingYears','YearsAtCompany', 'YearsInCurrentRole','YearsWithCurrManager']

Checting for categorical data

In [ ]:
C_data =train_df.select_dtypes(include='object').columns.tolist()

In [ ]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
for i in C_data:
  train_df[i]= label_encoder.fit_transform(train_df[i])


In [ ]:
train_df.info()

We can now see that all the values have been converted to numberic value

# Visualualising

In [ ]:
import plotly.express as px

px.bar(train_df,'BusinessTravel','Attrition')

In [ ]:
px.bar(train_df,'Gender','Attrition')

In [ ]:
px.bar(train_df,'JobRole','Attrition')

In [ ]:
px.bar(train_df,'MaritalStatus','Attrition')

In [ ]:
px.bar(train_df,'Over18','Attrition')

In [ ]:
px.bar(train_df,'OverTime','Attrition')

For categorical columns ['OverTime','MaritalStatus','JobRole','Gender','BusinessTravel'] we can see  are important

In [ ]:
C_cols =['OverTime','MaritalStatus','JobRole','Gender','BusinessTravel']

Now we got the numerical col & categorical colums now let us train them

In [ ]:
Train = train_df[input_col + C_cols]
Test = train_df['Attrition']

# Training Model

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test,y_train, y_test = train_test_split(Train,Test , random_state=104,test_size=0.25, shuffle=True)

# Linear regression

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
LR = LinearRegression().fit(X_train, y_train)
preX = LR.predict(X_train)
preY = LR.predict(X_test)
print(mean_squared_error(y_train, preX)," ",mean_squared_error(y_test, preY))

0.08811546583641058   0.09986387146069242


# DecisionTreeRegressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor

DTR = DecisionTreeRegressor(random_state=42,max_leaf_nodes= 4,max_depth =3,max_features=16,).fit(X_train, y_train)

preX = DTR.predict(X_train)
preY = DTR.predict(X_test)
print(mean_squared_error(y_train, preX)," ",mean_squared_error(y_test, preY))

0.09087558611137345   0.09382993829763928


# RandomForestRegressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
  

RFT = RandomForestRegressor( random_state = 42, n_estimators=350,max_depth= 6,min_samples_split=20,max_features=8,max_leaf_nodes=22).fit(X_train, y_train)
preX = RFT.predict(X_train)
preY = RFT.predict(X_test)
print(mean_squared_error(y_train, preX)," ",mean_squared_error(y_test, preY))

0.06562061550032892   0.08851652499556996


# XGBRegressor

In [ ]:
from xgboost import XGBRegressor
XGB = XGBRegressor(random_state=42, n_jobs=-1).fit(X_train, y_train)
preX = RFT.predict(X_train)
preY = RFT.predict(X_test)
print(mean_squared_error(y_train, preX)," ",mean_squared_error(y_test, preY))

[11:33:17] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.01245719968178202   0.0943242857142857


# Parameter Tuning

parameter tuner help change the model type and change the values and try to get  best value

In [ ]:
T_v = []
V_v = []
x=0
for i in range(2,30,2):
    mod = XGBRegressor( random_state = 42, n_estimators=40,max_depth = i).fit(X_train, y_train)
    preX = mod.predict(X_train)
    preY = mod.predict(X_test)
    print(mean_squared_error(y_train, preX)," ",mean_squared_error(y_test, preY),i,x)
    T_v.append(mean_squared_error(y_train, preX))
    V_v.append(mean_squared_error(y_test, preY))
    x=x+1

[12:35:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.07722730911824977   0.09064874179661375 2 0
[12:35:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.05026902148039129   0.08705607842989564 4 1
[12:35:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.02077541510057447   0.0942107132199863 6 2
[12:35:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.00690651339621614   0.10690822634010255 8 3
[12:35:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.0017933380044674234   0.11511934747267903 10 4
[12:35:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.0007

00.0898180531874654

In [ ]:
import plotly.express as px
px.line(V_v)

# Sub

In [217]:
Test_df = pd.read_csv("/content/playground-series-s3e3/test.csv")

In [221]:
input_col = ['Age','Education','EnvironmentSatisfaction','JobInvolvement', 'JobLevel','JobSatisfaction','MonthlyIncome','RelationshipSatisfaction', 'StockOptionLevel','TotalWorkingYears','YearsAtCompany', 'YearsInCurrentRole','YearsWithCurrManager']
C_cols =['OverTime','MaritalStatus','JobRole','Gender','BusinessTravel']
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
for i in C_cols:
  Test_df[i]= label_encoder.fit_transform(Test_df[i])
Train = Test_df[input_col + C_cols]
preX = RFT.predict(Train)
preX.shape

(1119,)

In [223]:
sub = pd.read_csv("/content/playground-series-s3e3/sample_submission.csv")
sub['Attrition'] = preX
pd.options.display.float_format = '{:,.3f}'.format
sub.to_csv('submission.csv', index=False)
from IPython.display import FileLink
FileLink('submission.csv') # Doesn't work on Colab, use the file browser instead to download the file.

/content/submission.csv

refrences:
1.https://jovian.com/learn/machine-learning-with-python-zero-to-gbms/lesson/gradient-boosting-with-xgboost
2.https://scikit-learn.org/stable/supervised_learning.html
